In [ ]:
GITHUB_USERNAME = "$GITHUB_USERNAME$"
GITHUB_REF = "$GITHUB_REF$"
NOTEBOOK_TYPE = "$NOTEBOOK_TYPE$"
PYTHON_VERSION = "$PYTHON_VERSION$"
IPYTHON_VERSION = "$IPYTHON_VERSION$"

In [ ]:
from pathlib import Path

import requests


if NOTEBOOK_TYPE == 'colab':
    # utils module doesn't exist on colab VM, so get current version from GitHub
    utils_module = Path('utils.py').resolve()
    response = requests.get(f'https://raw.githubusercontent.com/{GITHUB_USERNAME}/davos/{GITHUB_REF}/tests/utils.py')
    utils_module.write_text(response.text)
    # also need to install davos locally
    from utils import install_davos
    install_davos(source='github', ref=GITHUB_REF, fork=GITHUB_USERNAME)

In [ ]:
from contextlib import redirect_stdout
from io import StringIO
from subprocess import CalledProcessError
from textwrap import dedent

import davos
import IPython

from utils import mark, raises, run_tests

# tests for `davos.implementations.ipython_common`

In [ ]:
def test_ipython_common_imports():
    """
    check that functions that should've been imported from the 
    ipython_common module came from the right place
    """
    ipy_common_funcs = (
        '_check_conda_avail_helper', 
        '_run_shell_command_helper',
        '_set_custom_showsyntaxerror',
        '_showsyntaxerror_davos'
    )
    for func_name in ipy_common_funcs:
        func_obj = getattr(davos.implementations, func_name)
        func_module = getattr(func_obj, '__module__')
        assert func_module == 'davos.implementations.ipython_common', (
            f"davos.implementations.{func_name} is {func_module}.{func_name}. "
            f"Expected davos.implementations.ipython_common.{func_name}"
        )

In [ ]:
@mark.jupyter
def test_check_conda_avail_helper():
    """test helper function for getting conda-related config fields"""
    expected_env_path = "/usr/share/miniconda/envs/kernel-env"
    # only part of output that matters is line with environment path
    expected_first_line = f"# packages in environment at {expected_env_path}:"
    
    result_output = davos.implementations.ipython_common._check_conda_avail_helper()
    result_first_line = result_output.splitlines()[0]
    result_env_path = result_first_line.split()[-1].rstrip(':')
    
    assert result_env_path == expected_env_path, (
        f"Result:\n\n{result_env_path}\n\nExpected:\n\n{expected_env_path}"
    )

In [ ]:
def test_run_shell_command_helper():
    """test helper function for davos.core.core.run_shell_command"""
    # this command should pass...
    with redirect_stdout(StringIO()) as tmp_stdout:
        davos.implementations.ipython_common._run_shell_command_helper('echo "test"')
        stdout = tmp_stdout.getvalue().strip()
    assert stdout == 'test', stdout
    
    # ...this command should fail
    with raises(CalledProcessError), redirect_stdout(StringIO()):
        davos.implementations.ipython_common._run_shell_command_helper('"tset " ohce')

In [ ]:
def test_set_custom_showsyntaxerror():
    """
    check that the IPython shell's .showsyntaxerror() method was 
    replaced with the custom davos implementation, and that the original 
    is stored in the davos config
    """
    orig_func = davos.implementations.ipython_common._showsyntaxerror_davos
    bound_method = get_ipython().showsyntaxerror
    unbound_func = bound_method.__func__
    assert unbound_func is orig_func, (
        f"{unbound_func.__module__}.{unbound_func.__name__}"
    )
    
    orig_method = davos.config._ipy_showsyntaxerror_orig
    assert orig_method is not None
    orig_qualname = f"{orig_method.__module__}.{origiorig_methodnal_method.__qualname__}"
    expected_orig_qualname = "IPython.core.interactiveshell.InteractiveShell.showsyntaxerror"
    assert orig_qualname == expected_orig_qualname, orig_qualname

In [ ]:
run_tests()